In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('/Users/ijeonghun/Documents/projects/dacon/웹로그기반/train.csv')

In [3]:
train.head(3)

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,3.0,Chrome,Windows,desktop,1,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,1.0,Samsung Internet,Android,mobile,1,1.0,0.0,1,0.0,0.0,Asia,Southeast Asia,Malaysia,(direct),(none),NaN,NaN


In [4]:
import sklearn.preprocessing as skpre #실제 전처리
import sklearn.impute as skimp # nan값 처리
import sklearn.pipeline as skpip
import sklearn.compose as skcom # 데이터 타입이 달라도 합치기 위함

In [5]:
import numpy as np

## 1. 컬럼 분류

In [6]:
target="TARGET"
num_cols=train.select_dtypes(np.number).columns.difference([target])
obj_cols=train.select_dtypes("object").columns.difference(['sessionID'])
drop_cols='sessionID'

In [7]:
train["transaction"].value_counts()

transaction
0.0     250081
1.0       2162
2.0         38
3.0          3
4.0          2
5.0          1
7.0          1
12.0         1
Name: count, dtype: int64

In [8]:
num_cols

Index(['bounced', 'duration', 'new', 'quality', 'transaction',
       'transaction_revenue'],
      dtype='object')

## 2. 전처리 과정 세분화

### 1. numeric

In [9]:
num_pip=skpip.make_pipeline(
    skimp.SimpleImputer(strategy="median"),
    skpre.StandardScaler()
    )

In [10]:
num_pip.fit(train[num_cols])

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [11]:
num_pip.transform(train[num_cols])

array([[-0.98742296,  2.1938646 , -1.74112841,  3.71320625, -0.08949396,
        -0.04258737],
       [-0.98742296, -0.22396425,  0.57434018, -0.25397202, -0.08949396,
        -0.04258737],
       [ 1.01273723, -0.34183341,  0.57434018, -0.25397202, -0.08949396,
        -0.04258737],
       ...,
       [-0.98742296, -0.13329567, -1.74112841, -0.16380888, -0.08949396,
        -0.04258737],
       [-0.98742296, -0.2572094 ,  0.57434018, -0.25397202, -0.08949396,
        -0.04258737],
       [-0.98742296, -0.10911738, -1.74112841,  0.10668055, -0.08949396,
        -0.04258737]])

### 2. Object

In [12]:
obj_pip=skpip.make_pipeline(
    skimp.SimpleImputer(strategy="most_frequent"),
    skpre.OneHotEncoder(handle_unknown="ignore",drop="first",sparse_output=False)
)

In [13]:
obj_pip.fit(train[obj_cols])

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(drop='first', handle_unknown='ignore',
                               sparse_output=False))])

## 3. 합치기

In [14]:
total_pip=skcom.make_column_transformer(
    (num_pip,num_cols),
    (obj_pip,obj_cols),
    remainder="passthrough"
)

In [15]:
total_pip

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['bounced', 'duration', 'new', 'quality', 'transaction',
       'transaction_revenue'],
      dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['OS', 'browser', 'continent', 'country', 'device', 'keyword',
       'referral_path', 'subcontinent', 'traffic_medium', 'traffic_source',
       'userID'],
      dtype='object'))])

In [16]:
total_pip.fit(train.head(5000))

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['bounced', 'duration', 'new', 'quality', 'transaction',
       'transaction_revenue'],
      dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['OS', 'browser', 'continent', 'country', 'device', 'keyword',
       'referral_path', 'subcontinent', 'traffic_medium', 'traffic_source',
       'userID'],
      dtype='object'))])

In [17]:
total_pip.transform(train.head(5000))

array([[-1.0610609502455806, 2.361818884833031, -1.756407504043819, ...,
        0.0, 'SESSION_000000', 17.0],
       [-1.0610609502455806, -0.21106736688091665, 0.5693439578786109,
        ..., 0.0, 'SESSION_000001', 3.0],
       [0.9424529286169205, -0.3364955716519716, 0.5693439578786109, ...,
        0.0, 'SESSION_000002', 1.0],
       ...,
       [0.9424529286169205, -0.3364955716519716, -1.756407504043819, ...,
        0.0, 'SESSION_004997', 1.0],
       [0.9424529286169205, -0.3364955716519716, 0.5693439578786109, ...,
        0.0, 'SESSION_004998', 1.0],
       [0.9424529286169205, -0.3364955716519716, 0.5693439578786109, ...,
        1.0, 'SESSION_004999', 1.0]], dtype=object)

## 4. 컬럼 이름

In [22]:
def column_name(x):
    x=x.replace("pipeline-1__","")
    x=x.replace("pipeline-2__","")
    x=x.replace("remainder__","")
    return x

In [24]:
col_names=list(map(column_name,total_pip.get_feature_names_out()))

In [25]:
pre_train=pd.DataFrame(total_pip.transform(train.head(5000)),columns=col_names)

In [26]:
pre_train

,bounced,duration,new,quality,transaction,transaction_revenue,OS_Android,OS_BlackBerry,OS_Chrome OS,OS_Linux,...,userID_USER_004493,userID_USER_004494,userID_USER_004495,userID_USER_004496,userID_USER_004497,userID_USER_004498,userID_USER_004499,userID_USER_004500,sessionID,TARGET
0,-1.061061,2.361819,-1.756408,4.688281,-0.070888,-0.029319,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SESSION_000000,17.0
1,-1.061061,-0.211067,0.569344,-0.228519,-0.070888,-0.029319,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SESSION_000001,3.0
2,0.942453,-0.336496,0.569344,-0.228519,-0.070888,-0.029319,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SESSION_000002,1.0
3,0.942453,-0.336496,0.569344,-0.228519,-0.070888,-0.029319,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SESSION_000003,1.0
4,0.942453,-0.336496,-1.756408,-0.228519,-0.070888,-0.029319,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SESSION_000004,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-1.061061,-0.313983,0.569344,-0.228519,-0.070888,-0.029319,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,SESSION_004995,2.0
4996,0.942453,-0.336496,0.569344,-0.228519,-0.070888,-0.029319,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,SESSION_004996,1.0
4997,0.942453,-0.336496,-1.756408,-0.228519,-0.070888,-0.029319,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,SESSION_004997,1.0
4998,0.942453,-0.336496,0.569344,-0.228519,-0.070888,-0.029319,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,SESSION_004998,1.0
